# **從簡單的資料集開始訓練模型**
此份程式碼會從簡單的二維資料集介紹完整深度學習模型的訓練流程，從模型建置、模型訓練、模型評估，至模型儲存、載入重現結果。

## 本章節內容大綱
* ### [創建資料集／載入資料集（Dataset Creating/ Loading）](#DatasetCreating/Loading)
* ### [模型建置（Model Building）](#ModelBuilding)
* ### [模型訓練（Model Training）](#ModelTraining)
* ### [模型評估（Model Evaluation）](#ModelEvaluation)
* ### [模型儲存／載入（Model Saving/ Loading）](#ModelSaving/Loading)
-----------------

## 匯入套件

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

<a name="DatasetCreating/Loading"></a>
## 創建資料集／載入資料集（Dataset Creating / Loading）

In [ ]:
np.random.seed(12)
num_samples_per_class = 1000

# 創建負樣本 (X1_neg, X2_neg)
negative_samples = np.random.multivariate_normal(
    mean=[0, 3],  # 各維度的平均值
    cov=[[1, 0.5], [0.5, 1]],  # 各維度的共變異數
    size=num_samples_per_class)  # 樣本數量

# 創建正樣本 (X1_pos, X2_pos)
positive_samples = np.random.multivariate_normal(
    mean=[3, 0],  # 各維度的平均值
    cov=[[1, 0.5], [0.5, 1]],  # 各維度的共變異數
    size=num_samples_per_class)  # 樣本數量

print('shape of neg samples:', negative_samples.shape)
print('shape of pos samples:', positive_samples.shape)

In [ ]:
inputs = np.vstack((negative_samples, positive_samples)).astype(np.float32)
targets = np.vstack((np.zeros((num_samples_per_class, 1),  # 負樣本標籤
                              dtype='float32'),
                     np.ones((num_samples_per_class, 1),  # 正樣本標籤
                             dtype='float32')))

In [ ]:
# 建立二維及三維的比較圖
plt.figure(figsize=(10, 4))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122, projection='3d')

'''Plot on 2-dimension space'''
# 繪製訓練資料集
ax1.scatter(negative_samples[:, 0],
            negative_samples[:, 1],
            label='negative samples')
ax1.scatter(positive_samples[:, 0],
            positive_samples[:, 1],
            label='negative samples')

ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax1.set_title('x1-x2 plane')

'''Plot on 3-dimensions space'''
# 繪製訓練資料集
ax2.scatter(negative_samples[:, 0],
            negative_samples[:, 1],
            np.zeros((num_samples_per_class, 1), dtype='float32'),
            label='negative samples')

ax2.scatter(positive_samples[:, 0],
            positive_samples[:, 1],
            np.ones((num_samples_per_class, 1), dtype='float32'),
            label='positive samples')

ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax2.set_zlabel('y')
ax2.set_title('x1-x2-y space')
ax2.view_init(45, 285)
plt.show()

In [ ]:
# 打亂資料集順序
shuffle_idx = tf.random.shuffle(range(2*num_samples_per_class), seed=17)
inputs = inputs[shuffle_idx]
targets = targets[shuffle_idx]

<a name="ModelBuilding"></a>
## 模型建置（Model Building）

目標：找到一個平面可以擬合這兩群資料點，假設此平面方程式為下列式子

![](https://i.imgur.com/6LVSuBR.png)

* ### Sequential model（序列模型）
單輸入單輸出的模型，依順序堆疊網路層。

In [ ]:
keras.backend.clear_session()  # 重置 keras 的所有狀態
tf.random.set_seed(17)  # 設定 tensorflow 隨機種子

model = keras.models.Sequential()
model.add(layers.Dense(1,  # 神經元個數
                       input_shape=inputs[0].shape))  # 輸入形狀

# 以下寫法等同以上結果，將所有網路層按順序，以串列(list)的方式輸進 Sequential
# model = keras.models.Sequential(
#     [layers.Dense(1, input_shape=inputs[0].shape)])

In [ ]:
model.summary()

* ### Functional API
除了單輸入單輸出外，也可以支援多輸入多輸出，相較 Sequential model 更彈性，依照變數傳遞的方式串接網路層。

In [ ]:
keras.backend.clear_session()  # 重置 keras 的所有狀態
tf.random.set_seed(17)  # 設定 tensorflow 隨機種子

x_inputs = layers.Input(shape=inputs[0].shape)
x_outputs = layers.Dense(1)(x_inputs)

model = keras.models.Model(inputs=x_inputs, outputs=x_outputs)

In [ ]:
model.summary()

<a name="ModelTraining"></a>
## 模型訓練（Model Training）

* ### 模型編譯（model compile）
設定模型訓練時，所需的優化器 (optimizer)、損失函數 (loss function)

In [ ]:
model.compile(optimizer='rmsprop',        # 優化器
              loss='mean_squared_error')  # 損失函數

In [ ]:
history = model.fit(
    inputs,                # 輸入（訓練集）
    targets,               # 標籤（訓練集）
    batch_size=16,         # 批次數量
    epochs=20,             # 訓練回合數
    validation_split=0.2)  # 切分驗證集，前 80 % 為訓練集，後 20 % 為驗證集

<a name="ModelEvaluation"></a>
## 模型評估（Model Evaluation）

* ### 視覺化訓練過程的評估指標 （Visualization）

In [ ]:
# type(history.history) = dictionary
print(history.history.keys())

In [ ]:
train_loss = history.history['loss']
valid_loss = history.history['val_loss']

In [ ]:
# 繪製 Epochs vs. MSE
plt.figure(figsize=(15, 4))
plt.plot(range(len(train_loss)), train_loss, label='train_loss')
plt.plot(range(len(valid_loss)), valid_loss, label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.show()

* ### 模型預測（Model predictions）

In [ ]:
predictions = model(inputs)
print(predictions)
print(type(predictions))

In [ ]:
predictions = model.predict(inputs)
print(predictions)
print('Type:', type(predictions))

In [ ]:
loss = model.evaluate(inputs, targets)
print(f'MSE: {loss}')

* ### 視覺化結果

In [ ]:
model.variables  # 模型變數

In [ ]:
w = model.variables[0]
b = model.variables[1]

In [ ]:
# 建立二維及三維的比較圖
plt.figure(figsize=(10, 4))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122, projection='3d')

'''Plot on 2-dimension space'''
# 決策邊界函數為 w1*x1 + w2*x2 + b = 0.5
x = np.linspace(-3, 6, 100)  # 從 -3 到 6 切分 100 等分
boundary = - w[0] / w[1] * x + (0.5 - b) / w[1]

# 繪製決策邊界線
ax1.plot(x, boundary, '-r', label='Decision Boundary')

# 繪製訓練資料集
ax1.scatter(negative_samples[:, 0],
            negative_samples[:, 1],
            label='negative samples')
ax1.scatter(positive_samples[:, 0],
            positive_samples[:, 1],
            label='negative samples')

ax1.set_xlabel('x1')
ax1.set_ylabel('x2')
ax1.set_title('x1-x2 plane')

'''Plot on 3-dimensions space'''
x1 = np.linspace(-3, 5, 100)
x2 = np.linspace(-3, 5, 100)
x1, x2 = np.meshgrid(x1, x2)  # ［-3:5, -3:5］切分成 100x100 個位置點
y = w[0] * x1 + w[1] * x2 + b

ax2.contour3D(x1, x2, y, 100, alpha=0.5, cmap='viridis')  # 擬合平面
ax2.plot3D(x, boundary, 0.5, '-r', label='Decision Boundary')  # 決策邊界線

# 繪出訓練資料集
ax2.scatter(negative_samples[:, 0],
            negative_samples[:, 1],
            np.zeros((num_samples_per_class, 1), dtype='float32'),
            label='negative samples',
            depthshade=False)
ax2.scatter(positive_samples[:, 0],
            positive_samples[:, 1],
            np.ones((num_samples_per_class, 1), dtype='float32'),
            label='positive samples',
            depthshade=False)

ax2.set_zlim(0, 1)
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')
ax2.set_zlabel('y')
ax2.set_title('x1-x2-y space')
ax2.view_init(45, 285)
ax2.legend(bbox_to_anchor=(1.05, 1))

plt.show()

<a name="ModelSaving/Loading"></a>
## 模型儲存／載入（Model Saving/ Loading）

In [ ]:
model.save('Data/model.h5')  # 儲存位置

In [ ]:
new_model = keras.models.load_model('Data/model.h5')  # 讀取位置

In [ ]:
new_model.summary()

In [ ]:
loss= new_model.evaluate(inputs, targets)
print(f'MSE: {loss}')

----------------
## 動手試試看： 
1. 嘗試改動 random seed，觀察訓練的結果（收斂速度以及 MSE 表現等等）
2. 嘗試改動 batch_size，觀察訓練的結果（收斂速度以及 MSE 表現等等）